In [1]:
import os
import sys
import json
import math
import cachetools
import numpy as np
import pandas as pd
import configparser
from snowflake.snowpark import Session
from copy import copy
from snowflake.snowpark import Row
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col, lit, sql_expr, get, get_path, udf, udtf, table_function, sproc, seq8, uniform, when_matched, when_not_matched, cast, try_cast, asc, asc_nulls_first, asc_nulls_last, collate, startswith, endswith, equal_nan, is_null, in_, when
from snowflake.snowpark.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType, BooleanType, FloatType, PandasSeries, PandasSeriesType, PandasDataFrame, PandasDataFrameType
from snowflake.snowpark.exceptions import SnowparkJoinException, SnowparkSQLException
from snowflake.snowpark.files import SnowflakeFile
from snowflake.snowpark.column import METADATA_FILENAME, METADATA_FILE_ROW_NUMBER
from collections import Counter
from typing import Iterable, Tuple

# Read snowflake credentials securely
config = configparser.ConfigParser()
config.read('assets/credentials.cfg')

connection_parameters = dict(
   account   =  config['SNOWPARKAWS']['SNOWFLAKE_ACCOUNT'],
   user      =  config['SNOWPARKAWS']['SNOWFLAKE_USER'],
   password  =  config['SNOWPARKAWS']['SNOWFLAKE_PASSWORD'],
   role      =  config['SNOWPARKAWS']['SNOWFLAKE_ROLE'],  # optional
   warehouse =  config['SNOWPARKAWS']['SNOWFLAKE_WAREHOUSE'],  # optional
   database  =  config['SNOWPARKAWS']['SNOWFLAKE_DATABASE'],  # optional
   schema    =  config['SNOWPARKAWS']['SNOWFLAKE_SCHEMA'],  # optional
)

# Pass this dictionary to the Session.builder.configs method to return a builder object that has these connection parameters.
# Call the create method of the builder to establish the session.
session = Session.builder.configs(connection_parameters).create()

#### `First table is CUST_INFO and insert 1000 customers into it using this new Python UDTF.`

In [2]:
session.sql("""create or replace function gen_cust_info(num_records number)
returns table (custid number(10), cname varchar(100), spendlimit number(10,2))
language python
runtime_version=3.8
handler='CustTab'
packages = ('Faker')
as $$
from faker import Faker
import random
fake = Faker()
# Generate a list of customers  

class CustTab:
    # Generate multiple customer records
    def process(self, num_records):
        customer_id = 1000 # Starting customer ID                 
        for _ in range(num_records):
            custid = customer_id + 1
            cname = fake.name()
            spendlimit = round(random.uniform(1000, 10000),2)
            customer_id += 1
            yield (custid,cname,spendlimit)

$$;
""").collect()

[Row(status='Function GEN_CUST_INFO successfully created.')]

In [14]:
session.sql("show functions like '%_info' in schema demo_db.dt_demo").select(col('"catalog_name"'), col('"schema_name"'),col('"name"')).show()

-------------------------------------------------------------
|"catalog_name"  |"schema_name"  |"name"                    |
-------------------------------------------------------------
|DEMO_DB         |DT_DEMO        |GEN_CUST_INFO             |
|                |               |SYSTEM$TASK_RUNTIME_INFO  |
-------------------------------------------------------------



In [15]:
session.sql("""
create or replace table cust_info as select * from table(gen_cust_info(1000)) order by 1
""").collect()

[Row(status='Table CUST_INFO successfully created.')]

In [16]:
session.table("CUST_INFO").limit(10).to_pandas()

,CUSTID,CNAME,SPENDLIMIT
0,1626,Brooke Logan,5263.54
1,1627,Rebecca Fuentes,1916.42
2,1628,Cody Jackson,2503.86
3,1629,Glenda Rhodes,5280.69
4,1630,Christopher Shannon,8311.59
5,1631,Todd Davis,8748.66
6,1632,Clifford Brown,3328.38
7,1633,Anne Collins DDS,5700.27
8,1634,Gregory Lopez,3865.13
9,1635,Gregory Rivas,3853.00


#### `Next table is PROD_STOCK_INV and insert 100 products inventory into it using this new Python UDTF.`

In [18]:
session.sql("""create or replace function gen_prod_inv(num_records number)
returns table (pid number(10), pname varchar(100), stock number(10,2), stockdate date)
language python
runtime_version=3.10
handler='ProdTab'
packages = ('Faker')
as $$
from faker import Faker
import random
from datetime import datetime, timedelta
fake = Faker()

class ProdTab:
    # Generate multiple product records
    def process(self, num_records):
        product_id = 100 # Starting customer ID                 
        for _ in range(num_records):
            pid = product_id + 1
            pname = fake.catch_phrase()
            stock = round(random.uniform(500, 1000),0)
            # Get the current date
            current_date = datetime.now()
            
            # Calculate the maximum date (3 months from now)
            min_date = current_date - timedelta(days=90)
            
            # Generate a random date within the date range
            stockdate = fake.date_between_dates(min_date,current_date)

            product_id += 1
            yield (pid,pname,stock,stockdate)

$$;
""").collect()

[Row(status='Function GEN_PROD_INV successfully created.')]

In [19]:
session.sql(f"""create or replace table prod_stock_inv as select * from table(gen_prod_inv(100)) order by 1""").collect()

[Row(status='Table PROD_STOCK_INV successfully created.')]

In [20]:
session.table("PROD_STOCK_INV").limit(10).to_pandas()

,PID,PNAME,STOCK,STOCKDATE
0,188,Inverse radical data-warehouse,923.0,2023-10-21
1,189,Quality-focused motivating open system,564.0,2023-09-23
2,190,Organic multi-tasking workforce,593.0,2023-12-12
3,191,Re-contextualized global software,679.0,2023-11-30
4,192,Intuitive grid-enabled toolset,980.0,2023-11-17
5,193,Reverse-engineered zero administration support,582.0,2023-11-15
6,194,Public-key neutral core,738.0,2023-12-08
7,195,Stand-alone bandwidth-monitored time-frame,562.0,2023-11-11
8,196,Organic intermediate access,739.0,2023-10-11
9,197,Organic static database,984.0,2023-10-01


#### `Next table is SALESDATA to store raw product sales by customer and purchase date`

In [25]:
session.sql("""create or replace function gen_cust_purchase(num_records number, ndays number)
returns table(custid number(10), purchase variant)
language python
runtime_version=3.10
handler='genCustPurchase'
packages=('Faker')
as
$$
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

class genCustPurchase:
    # Generate multiple customer purchase records
    def process(self, num_records, ndays):
        for _ in range(num_records):
            c_id = fake.random_int(min=1001, max=1999)

            customer_purchase = {
                 'custid':c_id
                ,'purchased':[]
            }

            # Get the current date
            current_date = datetime.now()

            # Calculate the maximum date(days from now)
            min_date = current_date - timedelta(days=ndays)

            # Generate a random date within the date range
            pdate = fake.date_between_dates(min_date, current_date)

            purchase ={
                'prodid':fake.random_int(min=101, max=199)
               ,'quantity':fake.random_int(min=1, max=5)
               ,'purchase_amount':round(random.uniform(10, 1000),2)
               ,'purchase_date':pdate
            }

            customer_purchase['purchased'].append(purchase)

            yield(c_id, purchase)

$$;
""").collect()


[Row(status='Function GEN_CUST_PURCHASE successfully created.')]

In [26]:
# Create table and insert records
session.sql(f"""create or replace table salesdata as select * from table(gen_cust_purchase(10000,10))""").collect()

[Row(status='Table SALESDATA successfully created.')]

In [30]:
session.table("SALESDATA").show(10)

-----------------------------------------------
|"CUSTID"  |"PURCHASE"                        |
-----------------------------------------------
|1990      |{                                 |
|          |  "prodid": 196,                  |
|          |  "purchase_amount": 388.71,      |
|          |  "purchase_date": "2023-12-12",  |
|          |  "quantity": 5                   |
|          |}                                 |
|1827      |{                                 |
|          |  "prodid": 193,                  |
|          |  "purchase_amount": 57.87,       |
|          |  "purchase_date": "2023-12-15",  |
|          |  "quantity": 1                   |
|          |}                                 |
|1279      |{                                 |
|          |  "prodid": 197,                  |
|          |  "purchase_amount": 950.96,      |
|          |  "purchase_date": "2023-12-17",  |
|          |  "quantity": 3                   |
|          |}                           

In [34]:
session.sql("show tables in schema demo_db.dt_demo").show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"          |"database_name"  |"schema_name"  |"kind"  |"comment"  |"cluster_by"  |"rows"  |"bytes"  |"owner"   |"retention_time"  |"automatic_clustering"  |"change_tracking"  |"search_optimization"  |"search_optimization_progress"  |"search_optimization_bytes"  |"is_external"  |"enable_schema_evolution"  |"owner_role_type"  |"is_event"  |"budget"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
# Close Snowpark session
session.close()